# we downloaded 100 datasets from pangalodb..


In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

import matplotlib as mpl
mpl.rc("figure", dpi=200)

In [ ]:
# first we did this with njobs 10, then 2 to finish all the jobs..
#from ubergauss import tools as t 
#t.xmap(lambda x: loader.nuread('/home/ubuntu/data/scdata/',dataset= x,min_genes = 0 ), labels,n_jobs = 2)

In [ ]:
'''
# collection data 

we need: 
    - pfail+ufail
    - cells under 200 reads
    - # classlabels 
    - # okcells in total 
'''

import seaborn as sns
import pandas as pd
from cellsaw import io_utils as loader
from cellsaw.io_utils.loadadata import get100names,annotatepangalo
import lmz
import scanpy as sc
import numpy as np

labels = get100names()
shortlabels = [label[:label.index('_')] for label in labels]




def getdata(dataset):
    adata = loader.nuread('/home/ubuntu/data/scdata/',dataset= dataset,min_genes = 0 )    


    pfail = adata.obs['celltype'] == "no pangalo"
    ufail = adata.obs['celltype'] == "Unknown"
    classfail = np.logical_or(pfail,ufail)
    adata= adata[np.logical_not(classfail)]
    sc.pp.filter_cells(adata,min_counts = 200)
    
    
    return dataset,adata.X.shape[0], len(np.unique(adata.obs['celltype']))
    

data = lmz.Map(getdata, labels)
#getdata("Kaposissarcoma_10xchromium_SRA843432-SRS4322339_2914")


In [ ]:
'''
# next we have some filter criteria:

- need at least 2 celltypes 
- need at least 2000 cells 
- 200 genes per cell!!!!!!!!!
- we need at least 4 examples per tissuetype
'''

labels, types, values = lmz.Transpose(data)
df = {"dataset":labels, "cells": types, "celltypes": values}
df = pd.DataFrame(df)


df = df[ df['cells'] > 2000]
df = df[ df['celltypes'] > 1]

names = df['dataset'].to_list()
a,b = np.unique([c[:5] for c in names ], return_counts=True)
itemcounts = lmz.Zip(a,b)
shitlist =  [item for item, count in itemcounts if count < 4]
select = [True if c[:5] not in shitlist else False for c in names]

df = df[select]

In [ ]:
df['dataset'].to_list()